## 26 Esfera com wireframe (culling e teste de profundidade ativados)

Neste exemplo, uma malha triangular esférica (ou seja, 3D) é renderizada de diferentes formas para ilustrar os efeitos da ativação e desativação dos recursos de culling e de teste de profundidade. Também é utilizado múltiplos viewports para renderizar em diferentes partes da tela.

A primeira renderização (da esquerda para direita) é uma simples animação da esfera sendo rotacionada. Como foi utilizada uma cor sólida, quase não conseguimos perceber a rotação.

Na segunda renderização, foi adicionada a renderização do wireframe da esfera. Com isso, é possível perceber a rotação. Além disso, temos o efeito de uma esfera que gira para os dois lados. Isso acontece porque a renderização do wireframe é feita sobre a renderização da esfera sólida sem levar em consideração a profundidade dos triângulos. 

A terceira renderização é semelhante à segunda. A diferença é que o teste de profundidade está ativado. Agora, é possível percebe o lado para o qual a esfera gira. No entanto, as linhas do wireframe apresentam artefatos. Esse efeito é conhecido como z-fighting. Ele ocorre quando dois triângulos sobrepostos possuem profundidades muito similares, causando uma indecisão de qual triângulo está na frente. 

Na quarta renderização, o z-fighting é resolvido adicionando uma transformação de escala à transformação de modelo do wireframe a fim de fazê-lo ser um pouco maior que a esfera sólida.

Por fim, a quinta renderização mostra o efeito obtido ao ativar o recurso de culling. Este efeito acontece porque a esfera foi descrita considerando o espaço cartesiano (destro) e está sendo renderizada no espaço do OpenGL (canhoto). Então, o eixo z é invertido, e os triângulos na parte positiva do eixo z fica mais longe e vice-versa. Além disso, os triângulos do eixo z negativo no plano cartesiano possuem a face frontal apontando para o lado oposto do observador. E, por isso, não são renderizados apesar de terem ficado mais perto do observador depois da inversão do eixo z.

In [1]:
import glm
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v2 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v2 import ModelRenderer
from cg.models.SphereMesh_v1 import SphereMesh

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        # cria uma malha esférica
        sphere_mesh = SphereMesh(0.8, 20, 20)
        
        # cria o objeto responsável por carregar os dados para a GPU e renderizá-los
        self.sphereRenderer = ModelRenderer(sphere_mesh.getVertexPositions(), vertex_indices=sphere_mesh.getVertexIndices())
        
        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()
        
        # ativa o shader programa para configurar uma cor única para todos os vértices
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.release()

        # recupera o endereço da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        
        # configura os dados do modelo para serem os dados de entrada do shader program
        self.sphereRenderer.setVertexPositionLoc(position_loc)

        # inicializa a variável que contém o ângulo de rotação
        self.angle = 0
        
    def paintGL(self):
        
        # configura a cor de background
        gl.glClearColor(0, 0, 0, 1)
        
        # limpa o background com a cor especificada e o buffer de profundidade
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # incrementa a variável que contém o ângulo de rotação
        self.angle += 0.01
        
        # atualiza o viewport para o objeto ser renderizado mais a esquerda
        # renderiza a esfera
        gl.glViewport(0, 0, int(self.width / 5), self.height)
        self.renderSphere()
        
        # atualiza o viewport para o objeto ser renderizado do lato esquerdo
        # renderiza a esfera com wireframe
        gl.glViewport(int(self.width / 5), 0, int(self.width / 5), self.height)
        self.renderSphereWithWireframe()
        
        # atualiza o viewport para o objeto ser renderizado no centro
        # renderiza a esfera com wireframe (teste de profundidade ativado)
        gl.glViewport(int(2 * self.width / 5), 0, int(self.width / 5), self.height)
        self.renderSphereWithWireframeDepth()
        
        # atualiza o viewport para o objeto ser renderizado no lado direito
        # renderiza a esfera com wireframe escalado (teste de profundidade ativado)
        gl.glViewport(int(3 * self.width / 5), 0, int(self.width / 5), self.height)
        self.renderSphereWithScaledWireframeDepth()
        
        # atualiza o viewport para o objeto ser renderizado mais a direita
        # renderiza a esfera com wireframe escalado (teste de profundidade e culling ativado)
        gl.glViewport(int(4 * self.width / 5), 0, int(self.width / 5), self.height)
        self.renderSphereWithScaledWireframeDepthCulling()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
        
    def renderSphere(self):
        
        # calcula a matriz de rotação da esfera e atualiza a matriz do shader
        model_matrix = glm.rotate(glm.mat4(), glm.radians(self.angle), glm.vec3(0.0, 1.0, 0.0))
        self.shaderProgram.setUniformMVPMatrix(model_matrix)
        
        # mudar a cor no shader e renderiza a esfera
        self.shaderProgram.setUniformColor(np.array([0.0, 0.0, 0.5, 1.0], dtype=np.float32))
        self.sphereRenderer.render()
        
    def renderSphereWithWireframe(self):
        
        # renderiza a esfera
        self.renderSphere()
        
        # mudar a cor no shader e renderiza o wireframe da esfera
        self.shaderProgram.setUniformColor(np.array([1.0, 1.0, 1.0, 1.0], dtype=np.float32))
        self.sphereRenderer.renderWireframe()
        
    def renderSphereWithWireframeDepth(self):

        # ativa o teste de profundidade
        gl.glEnable(gl.GL_DEPTH_TEST);

        # renderiza a esfera com wireframe
        self.renderSphereWithWireframe()

        # desativa o teste de profundidade
        gl.glDisable(gl.GL_DEPTH_TEST);
    
    def renderSphereWithScaledWireframeDepth(self):
        
        # ativa o teste de profundidade
        gl.glEnable(gl.GL_DEPTH_TEST);

        # renderiza a esfera
        self.renderSphere()
        
        # calcula a matriz de transformação do wireframe para que ele rotacione igual a esfera e que seja um pouco maior
        # perceba que a matriz de rotação é passada como parâmetro para a função que gera a matriz de escala
        # nesse caso, a composição das matrizes é feita pelo próprio glm
        rot = glm.rotate(glm.mat4(), glm.radians(self.angle), glm.vec3(0.0, 1.0, 0.0))
        model_matrix = glm.scale(rot, glm.vec3(1.005, 1.005, 1.005))
        self.shaderProgram.setUniformMVPMatrix(model_matrix)
        
        # mudar a cor no shader e renderiza o wireframe da esfera
        self.shaderProgram.setUniformColor(np.array([1.0, 1.0, 1.0, 1.0], dtype=np.float32))
        self.sphereRenderer.renderWireframe()

        # desativa o teste de profundidade
        gl.glDisable(gl.GL_DEPTH_TEST);
        
    def renderSphereWithScaledWireframeDepthCulling(self):
        
        # ativa o recurso de culling
        gl.glEnable(gl.GL_CULL_FACE);

        self.renderSphereWithScaledWireframeDepth()

        # desativa o recurso de culling
        gl.glDisable(gl.GL_CULL_FACE);
        
    def resizeGL(self, width, height):

        # armazena a largura e a altura da janela
        self.width = width
        self.height = height

def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(1000, 300)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
! jupyter nbconvert --to python 26_Esfera_com_wireframe_culling_depth.ipynb
%run -i 26_Esfera_com_wireframe_culling_depth.py